In [1]:
import tenzing
from tenzing.core.models import model_relation
from tenzing.core.typesets import tenzing_typeset, tenzingTypeset
from tenzing.core.model_implementations import *
from collections import Counter
import pandas as pd
import numpy as np
from shapely import wkt
from IPython.core.display import display, HTML
%load_ext autoreload
%autoreload 2

In [2]:
x = tenzingTypeset([tenzing_float, tenzing_integer])
x.relation_map

{tenzing_integer: {},
 tenzing_float: {tenzing_integer: <tenzing.core.models.model_relation at 0x11b8ccc88>}}

In [3]:
from tenzing.core.model_implementations.typesets import tenzing_standard

df = pd.DataFrame({'a': range(3), 'b': [2 * i for i in range(3)], 'c': ['howdy', 'howdy', 'doody']})
x = tenzing_standard()
x.prep(df)
summer = x.summarize(df)


You forgot to implement handling for multiple matches. Go fix that retard


In [4]:
x = {1: 2}
x.pop(1)
x

{}

In [5]:
x = tenzing_standard()
x.prep(df)
y = x.summary_report(df)

#HTML(y.generate_html()['Overview'])
HTML(y.generate_html())

You forgot to implement handling for multiple matches. Go fix that retard


Number of Observations,3
Number of Variables,3
tenzing_integer,2
tenzing_string,1


In [14]:
y.column_summary

{'a': {'nunique': 3.0,
  'mean': 1.0,
  'std': 1.0,
  'max': 2.0,
  'min': 0.0,
  'median': 1.0,
  'n_records': 3,
  'n_zeros': 1,
  'perc_zeros': 0.3333333333333333,
  'na_count': 0,
  'perc_na': 0.0},
 'b': {'nunique': 3.0,
  'mean': 2.0,
  'std': 2.0,
  'max': 4.0,
  'min': 0.0,
  'median': 2.0,
  'n_records': 3,
  'n_zeros': 1,
  'perc_zeros': 0.3333333333333333,
  'na_count': 0,
  'perc_na': 0.0},
 'c': {'nunique': 2,
  'n_records': 3,
  'frequencies': {'howdy': 2, 'doody': 1},
  'na_count': 0,
  'perc_na': 0.0}}

In [6]:

from pandas_profiling import ProfileReport
ProfileReport(df)


Number of variables,3
Number of observations,3
Total Missing (%),0.0%
Total size in memory,152.0 B
Average record size in memory,50.7 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [7]:
from tenzing.core.models import tenzing_model

type(tenzing_model)

tenzing.utils.singleton.Singleton

In [8]:
tenzing_integer

tenzing_integer

In [9]:
import pandas as pd
import numpy as np
x = pd.Series([1,2,3, np.nan])

print(x in tenzing_integer)
print(x in tenzing_float)

True
False


In [10]:
int_float_relation = model_relation(tenzing_float, lambda x: False, lambda x: False)
tenzing_float.register_relation(int_float_relation)

In [11]:
import numpy as np

x = pd.Series(list(range(10000)))
%timeit x.astype(np.float)
%timeit x.astype(float)

96.6 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
90.4 µs ± 7.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
x.astype(int).nbytes

80000

In [13]:
test_series = pd.Series([pd.datetime(2010, 1, 1), pd.datetime(2010, 8, 2), pd.datetime(2011, 2, 1), np.nan])
tenzing_timestamp.summarize(test_series)

{'nunique': 3,
 'min': Timestamp('2010-01-01 00:00:00'),
 'max': Timestamp('2011-02-01 00:00:00'),
 'n_records': 4,
 'perc_unique': 1.0,
 'range': Timedelta('396 days 00:00:00'),
 'na_count': 1,
 'perc_na': 0.25}